In [13]:
import os
import re

In [127]:
#читаем файл
path =r"C:/Users/Xiaomi/Downloads/science"
os.chdir(path)
def peak_sum(file_path):
    x=[] #массив со значениями
    b = -1 #для определения перехода через 0    -1 или 1
    suma=0 #сумма как переменная
    sum_max=0 #по ней определяем пик
    peak =[] #пик как переменная
    peak_max=[] #наш пик
    x_mod=[]
    k=0 #кол-во в пике
    length=0 #кол-во строк
    r=3
    n=0 #для чистки
    with open(file_path, 'r') as f_r: #на чтение
        l = [line.strip() for line in f_r]
        for line in l:
            length+=1
            x+=[float(s) for s in re.findall(r'.\w+[.]\w+|\w+[.]\w+', line)] #набиваем массив значениями
            for i in range(3): #убираем три(?) последних значения
                x.pop(-1)
            r=3
            n=0
            for i in range(12): #чистим незначащие каналы
                x.pop(-r)
                n+=1
                if n ==2:
                    r+=n
                    n=0
    x_mod = padding(x,length)
#     for i in range(length):
#         for j in range(49): #по размеру bin_fixed
#             f_w.write(str(x_mod[j][i])+' ')
#         f_w.write(str('\n'))
    for i in range(100): #вроде как в первой сотне должно быть событие
        for j in range(49):
        #продолжаем или начинаем пик
            if x_mod[j][i]>0 and (b==-1 or b==1):
                b=1
                suma+=x_mod[j][i]
                peak.append(x_mod[j][i])
                k+=1
            #смена знака
            if x_mod[j][i]<0 and b==1:
                b=-1
                if suma > sum_max and k>1:
                    sum_max = suma + 0
                    peak_max = [float(s) for s in peak]
                peak*=0  
                k=0
                suma=0
    peak_max_buf = str(peak_max)
    f_w.write((peak_max_buf)+"\n")
def padding(arr,length):
    maximum = 0 #максимум в хвосте
    index_max = 0 #индекс максимума
    bin_fixed = 650 #известный бин к которому подбиваем
    sum_time = 0 #сумма с весом
    suma = 0 #обычная сумма
    index_centre = 0 #индекс центра масс
    line = [] #столбец
    line2=[] #буфер
    final = [] #результат (двумерный массив)
    ln = length #длина столбца
    for i in range(49):
        for j in range(length): #переписываем столбец в строку
            line.append(arr[49*j+i])
        for j in range(bin_fixed-55,bin_fixed-25): #ищем максимум
            if line[j]>maximum:
                maximum = line[j]
                index_max=j
        sum_time+=line[index_max]*index_max
        suma+=line[index_max]
        j = index_max-1
        while(line[j]>0): #вверх по каналу
            sum_time+=line[j]*j
            suma+=line[j]
            j-=1
        j = index_max+1
        while(line[j]>0): #вниз по каналу
            sum_time+=line[j]*j
            suma+=line[j]
            j+=1
        index_centre = int(sum_time/suma) #находим центр масс
        while(index_centre!=bin_fixed): #подбиваем
            if bin_fixed-index_centre>0:
                line.insert(length,0)
                index_centre+=1
                line.pop(0)
            if bin_fixed-index_centre<0:
                line.insert(0,0)
                index_centre-=1
                line.pop(-1)
        line2 = [float(s) for s in line]
        final.insert(i,line2) #забиваем в результат
        sum_time = 0
        suma = 0
        index_centre = 0
        maximum = 0
        index_max=0
        line*=0
        ln = length
    return(final)

In [128]:
f_w = open("peaks/peaks.txt", 'w') #на запись
for file in os.listdir():
    if file.endswith('.txt'):
        file_path =f"{path}/{file}"
    peak_sum(file_path)